In [ ]:
import time
from langgraph.types import Command
from langgraph.graph import END
from llm import llm_model
from pydantic import BaseModel,Field
from typing import List,Literal, Optional
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from Prompts import supervisor_prompt,PreprocessingPlanner_prompt,cleaner_prompt,Reporter_prompt,VISUALIZATION_PROMPT
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import AgentExecutor, create_tool_calling_agent
from Toolkit.Tools import python_repl_ast,eda_fact_sheet
from Guardrails.Preprocessing import StructuredPlanOutput
from Guardrails.cleaner import CleaningSummary  
from Guardrails.report import BusinessReport
from Guardrails.visualizer import VisualizationReport
from langchain.output_parsers import PydanticOutputParser,OutputFixingParser


class Router(BaseModel):
    next: Literal["PreprocessingPlanner_node","Cleaner_node","Reporter_node","visualizer_node",END]= Field(description="The next node to route to. Must be one of the available nodes.")
    reasoning: str = Field(description="A short reasoning for the decision made.")

class AgentStateModel(BaseModel):
    messages: Optional[List] = None
    Instructions: Optional[str] = None
    Analysis: Optional[List[dict]] = None
    clean: Optional[List[dict]] = None
    Report: Optional[List[dict]] = None
    Visualizations: Optional[List[dict]] = None
    Path: Optional[str] = None
    next: Optional[str] = None
    current_reasoning: Optional[str] = None   
class DataAnalystAgent:
    def __init__(self):
        self.llm_model = llm_model

    def supervisor_node(self,state:AgentStateModel) -> Command[Literal["PreprocessingPlanner_node","Cleaner_node","Reporter_node","visualizer_node", END]]:

        """
        The central router of the workflow.
        It evaluates the current state and the last message to decide the next action.
        This node is designed to be highly token-efficient by creating a lean summary of the state
        instead of passing the full, verbose state objects to the LLM.
        """

        print("**************************below is my state right after entering****************************")

        print(state)

        print("************************** SUPERVISOR: EVALUATING STATE ****************************")

        state_summary = (
        f"Current Workflow Status:\n"
        f"- Analysis Plan Generated: {'Yes' if state.Analysis else 'No'}\n"
        f"- Cleaning Plan Generated: {'Yes' if state.clean else 'No'}\n"
        f"- Report Generated: {'Yes' if state.Report else 'No'}\n"
        f"- Visualizations Generated: {'Yes' if state.Visualizations else 'No'}\n"
    )

        messages_for_llm = [
            SystemMessage(content=supervisor_prompt),
            HumanMessage(content=state_summary),
        ]

        if state.messages:
            last_message = state.messages[-1]
            # Add a prefix to clearly label the last message for the LLM
            last_message_content = f"Last Event:\nThe last node to run was '{last_message.name}'. It reported the following:\n---\n{last_message.content}\n---"
            messages_for_llm.append(HumanMessage(content=last_message_content))
            print(f"--- Attaching last event from '{last_message.name}' ---")
        else:
            # Handle the very first run where there are no messages
            messages_for_llm.append(HumanMessage(content="Last Event: None. This is the first step of the workflow."))

        messages_for_this_attempt = list(messages_for_llm)

        print("***********************Invoking LLM for routing decision************************")

        parser = PydanticOutputParser(pydantic_object=Router)

        fixing_parser = OutputFixingParser.from_llm(parser=parser, llm=self.llm_model)

        # Build the retry chain
        chain = self.llm_model | fixing_parser

        # Add retries
        max_attempts = 3
        attempt = 0
        error_msg = None
        response = None

        while attempt < max_attempts:
            attempt += 1
            print(f"--- Attempt {attempt} ---")

            # Compose messages for this attempt
            messages_for_this_attempt = list(messages_for_llm)

            if error_msg:
                # Inject previous error info to let LLM know what failed
                messages_for_this_attempt.append(HumanMessage(content=f"Previous attempt failed due to: {error_msg}. Please follow the schema strictly: {Router.model_json_schema()}"))

            try:
                response = chain.invoke(messages_for_this_attempt)
                break
            
            except Exception as e:
                error_msg = str(e)
                print(f"--- Error on attempt {attempt}: {error_msg} ---")
                # If last attempt, will exit loop and propagate error

        if response is None:
            # All retries failed, fallback error
            fallback_msg = f"All {max_attempts} attempts failed. Last error: {error_msg}"
            print(f"--- Supervisor node failed ---\n{fallback_msg}")
            return Command(
                goto="END",
                update={
                    "next": "END",
                    "current_reasoning": fallback_msg
                }
            )
        
        goto = response.next
        
        print("********************************this is my goto*************************")
        print(goto)
        
        print("********************************")
        print(response.reasoning)
            
        if goto == "END":
            goto = END 
            
        print("**************************below is my state****************************")
        print(state)
        
        return Command(goto=goto, update={'next': goto, 
                                        'current_reasoning': response.reasoning}
                    )

    def PreprocessingPlanner_node(self, state: AgentStateModel) -> Command[Literal['supervisor']]:

        print("*****************called PreprocessingPlanner node************")

        Instructions = state.Instructions

        parser = PydanticOutputParser(pydantic_object=StructuredPlanOutput)

        fixing_parser = OutputFixingParser.from_llm(parser=parser, llm=self.llm_model)

        task_prompt = (
        f"Find the instructions given by the user here : {Instructions} and follow this {PreprocessingPlanner_prompt} to the letter.modify in this path:{state.Path}"
    )
        print(f"--- Sending this direct task to the agent ---\n{task_prompt}\n---------------------------------------------")

        system_prompt = ChatPromptTemplate.from_messages([
        
        ("system",
         "You are a DataFrame analyzer. Your primary tool is `eda_fact_sheet`. "
         "First, call the tool to get data insights. Then, based on the tool's output, "
         "provide a final answer formatted as a JSON object containing the preprocessing plan and summaries."),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")        
    ])
        
        Analyzer_agent = create_tool_calling_agent(
            llm=self.llm_model,
            tools=[eda_fact_sheet],
            prompt=system_prompt
        )

        agent_executor = AgentExecutor(
            agent=Analyzer_agent,
            tools=[eda_fact_sheet],
            verbose=True,
            handle_parsing_errors=True,
            return_intermediate_steps=True
        )

        # 5. Wrap execution in a retry loop
        max_attempts = 3
        attempt = 0
        while attempt < max_attempts:
            attempt += 1
            try:
                result = agent_executor.invoke({
                    "input": task_prompt,
                })

                # Try parsing the final output
                final_output_string = result.get("output", "")
                parsed_output: StructuredPlanOutput = fixing_parser.parse(final_output_string)

                # Successfully parsed → extract plan and summary
                plan_dict = {"plan": [step.model_dump() for step in parsed_output.plan]}
                summary_str = f"{parsed_output.summary}\n{parsed_output.details}"

                # Update state and return
                return Command(
                    update={
                        "messages": [
                            AIMessage(content=summary_str, name="PreprocessingPlanner_node")
                        ],
                        "Analysis": [{"final_answer": plan_dict}]
                    },
                    goto="supervisor",
                )

            except Exception as e:
                error_msg = (
                    f"Attempt {attempt} failed due to error: {str(e)}. "
                    f"Please strictly follow the schema: {StructuredPlanOutput.model_json_schema()}"
                )
                print(f"--- Runtime/Parsing error encountered ---\n{error_msg}")
                # Inject error into prompt for next retry
                # Use an f-string to properly embed all variables
                task_prompt = f"The previous attempt failed with this error: {error_msg}. Please correct your tool usage and try again. Here is the original task:\n---\n{task_prompt}"

        # If all attempts fail, fallback to supervisor with error message
        return Command(
            update={
                "messages": [
                    AIMessage(content="Error: The analysis agent failed to produce a valid preprocessing plan after multiple attempts.", 
                            name="Analyzer_node_Error")
                ],
                "Analysis": [{"error": "Parsing failed after retries"}]
            },
            goto="supervisor",
        )
                               
    def Cleaner_node(self, state: AgentStateModel) -> Command[Literal['supervisor']]:

        print("*****************called cleaner node************")

        Path = state.Path

        preprocessing_plan = state.Analysis[0]['final_answer']['plan']

        batched_plan = []
        current_batch = []

        for column_action in preprocessing_plan:
            # Add the current item to the batch
            current_batch.append(column_action)
            
            # If the batch is now full, add it to our final list and reset it
            if len(current_batch) == 4:
                batched_plan.append(current_batch)
                current_batch = []

            # After the loop, check if there are any leftover items in the last batch
        if current_batch:
          batched_plan.append(current_batch)
    
        parser = PydanticOutputParser(pydantic_object=CleaningSummary)

        fixing_parser = OutputFixingParser.from_llm(parser=parser, llm=self.llm_model)

        system_prompt = ChatPromptTemplate.from_messages(
        [(
            "system",
            "Follow the instructions here : {cleaner_prompt} and  in the input to the letter and make the necessary changes to the dataframe."
        ),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")        
        ]
        )
        
        Cleaner_agent = create_tool_calling_agent(
            llm=self.llm_model,
            tools=[python_repl_ast],
            prompt=system_prompt
        )

        agent_executor = AgentExecutor(
            agent=Cleaner_agent,
            tools=[python_repl_ast],
            verbose=True,
            # handle_parsing_errors=True,
            # return_intermediate_steps=True
        )

        # 3. Loop Through Batches and Invoke the Agent for Each
        all_batch_results = []
        final_clean_outputs = []

        for i, batch in enumerate(batched_plan, start=1):
            clean_plan_str = str(batch)
            task_prompt = (
                f"Apply the following cleaning plan (batch {i} of {len(batched_plan)}) to the dataset at path: {Path}\n"
                f"Plan details:\n{clean_plan_str}"
            )
            
            print(f"--- Sending task for Batch {i} to the agent ---\n{task_prompt}\n---------------------------------------------")

            max_attempts = 3
            attempt = 0
            batch_success = False
            while attempt < max_attempts:
                attempt += 1
                try:
                    result = agent_executor.invoke({"input": task_prompt,"cleaner_prompt":cleaner_prompt,"agent_scratchpad": [] })

                    final_output_string = result.get("output", "")
                    
                    # Try parsing the output
                    parsed_output: CleaningSummary = fixing_parser.parse(final_output_string)

                    # On success, store results and break the retry loop
                    all_batch_results.append(parsed_output)
                    final_clean_outputs.append({"final_answer": final_output_string})
                    batch_success = True
                    print(f"--- Batch {i} successful on attempt {attempt} ---")
                    time.sleep(10)
                    break

                except Exception as e:
                    error_msg = (
                        f"Attempt {attempt} for batch {i} failed with an error: {str(e)}. "
                        "You MUST provide a final answer that is a valid JSON object. Please review the plan and strictly follow the schema."
                    )
                    print(f"--- Runtime/Parsing error for Batch {i} ---\n{error_msg}")
                    # Inject error context for the next retry attempt on this specific batch
                    task_prompt = f"Your previous attempt failed with this error: {error_msg}\n\nPlease re-execute the original plan:\n{task_prompt}"

            # If a batch fails after all retries, exit the entire node with an error
            if not batch_success:
                error_message = f"Error: The cleaner agent failed to process batch {i} after {max_attempts} attempts."
                return Command(
                    update={
                        "messages": [AIMessage(content=error_message, name="Cleaner_node_Error")],
                        "clean": [{"error": f"Processing failed at batch {i}"}]
                    },
                    goto="supervisor",
                )

        final_summary = "All cleaning batches completed successfully.\n\n"
        for idx, summary in enumerate(all_batch_results, start=1):
            final_summary += f"--- Batch {idx} Summary ---\nSummary: {summary.summary}\nDetails: {summary.details}\n\n"

        return Command(
            update={
                "messages": [AIMessage(content=final_summary.strip(), name="cleaner_node")],
                "clean": final_clean_outputs,
            },
            goto="supervisor",
        )

    def Reporter_node(self, state: AgentStateModel) -> Command[Literal['supervisor']]:
        print("*****************called Reporter node************")

        Instructions = state.Instructions

        df_path = state.Path

        # --- STEP 1: Perform Reconnaissance MANUALLY (and only once) ---
        print("--- Reporter: Performing initial data reconnaissance with eda_fact_sheet ---")
        try:
            recon_result_str = str(eda_fact_sheet.run(path=df_path))
        except Exception as e:
            return Command(update={"messages": [AIMessage(content=f"Error during initial data recon: {e}", name="Reporter_node_Error")]}, goto="supervisor")
        
        print(f"--- Reporter: Condensing {len(recon_result_str)} characters of context... ---")
        condensation_prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a data analysis assistant. Summarize the following verbose JSON data profile into a concise, human-readable format for another AI agent to use. Focus on column names, data types, and key stats."),
            ("human", "Please summarize this data profile:\n\n{profile}")
        ])
        summarizer_chain = condensation_prompt | self.llm_model

        condensed_summary = summarizer_chain.invoke({"profile": recon_result_str}).content

        print(f"--- Reporter: Condensed Summary Created ---")

        # 1. Instantiate the parser for our new structured output
        parser = PydanticOutputParser(pydantic_object=BusinessReport)

        fixing_parser = OutputFixingParser.from_llm(parser=parser, llm=self.llm_model)

        task_prompt = (f"User Instructions: {Instructions}\n\nHere is a condensed summary of the dataset: \n---\n{condensed_summary}\n---\n\nNow, follow your main instructions: {Reporter_prompt}")

        print(f"--- Reporter: Sending condensed task to the main analysis agent ---")

        print(f"--- Sending this direct task to the agent ---\n{task_prompt}\n---------------------------------------------")

        system_prompt = ChatPromptTemplate.from_messages([
            ("system",
             "You are a Business Intelligence consultant. You have access to `eda_fact_sheet(df_path)` for initial recon and `python_repl_ast(query)` for deep-dive analysis. "
             "The CSV is at this path: {df_path}. "
             "Your mission is to autonomously analyze the data and produce a strategic business report. "),
            ("human", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])

        Reporter_agent = create_tool_calling_agent(
            llm=self.llm_model,
            tools=[python_repl_ast],
            prompt=system_prompt
        )

        agent_executor = AgentExecutor(
            agent=Reporter_agent,
            tools=[python_repl_ast],
            verbose=True,
            # handle_parsing_errors=True,
            # return_intermediate_steps=True
        )

        # 2. Wrap execution in a retry loop for robust parsing
        max_attempts = 3
        attempt = 0
        while attempt < max_attempts:
            attempt += 1
            try:
                result = agent_executor.invoke({
                    "input": task_prompt,
                    "df_path": df_path,
                })

                final_output_string = result.get("output", "")

                parsed_output: BusinessReport = fixing_parser.parse(final_output_string)

                summary_str = f"{parsed_output.subject}\n\n{parsed_output.executive_summary}"

                return Command(
                    update={
                        "messages": [AIMessage(content=summary_str, name="Reporter_node")],
                        "Report": [{"final_answer": parsed_output.model_dump()}] 
                    },
                    goto="supervisor",
                )

            except Exception as e:
                error_msg = (
                    f"Attempt {attempt} failed due to a parsing error: {str(e)}. "
                    f"You MUST provide a final answer that is a valid JSON object. Please strictly follow this schema: "
                    f"{BusinessReport.model_json_schema()}"
                )
                print(f"--- Runtime/Parsing error encountered ---\n{error_msg}")
                
                task_prompt =f"here is the error from the previous execution: {error_msg} so process the workflow accordingly" + task_prompt 

        
        return Command(
            update={
                "messages": [
                    AIMessage(content="Error: The reporter agent failed to produce a valid business report after multiple attempts.",
                              name="Reporter_node_Error")
                ],
                "Report": [{"error": "Parsing failed after all retries"}]
            },
            goto="supervisor",
        )


    def visualizer_node(self, state: AgentStateModel) -> Command[Literal['supervisor']]:
        """
        This node directs an agent to perform EDA and generate 10 business-focused visualizations.
        It enforces a structured JSON output for the final report and includes retry logic for parsing.
        """
        print("***************** called Visualizer node ************")

        df_path = state.Path

        print("--- Visualizer: Performing initial data reconnaissance... ---")
        try:
            recon_result_str = str(eda_fact_sheet.run(path=df_path))
        except Exception as e:
            return Command(update={"messages": [AIMessage(content=f"Error during initial data recon: {e}", name="Visualizer_node_Error")]}, goto="supervisor")

        print(f"--- Visualizer: Condensing {len(recon_result_str)} characters of context... ---")
        condensation_prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a data analysis assistant. Summarize the following verbose JSON data profile into a concise, human-readable format for another AI agent to use for creating visualizations."),
            ("human", "Please summarize this data profile:\n\n{profile}")
        ])
        summarizer_chain = condensation_prompt | self.llm_model

        condensed_summary = summarizer_chain.invoke({"profile": recon_result_str}).content

        print(f"--- Visualizer: Condensed Summary Created ---")

        parser = PydanticOutputParser(pydantic_object=VisualizationReport)

        fixing_parser = OutputFixingParser.from_llm(parser=parser, llm=self.llm_model)

        task_prompt = (f"Based on this data summary:\n---\n{condensed_summary}\n---\n\nNow, follow your main instructions to create visualizations: {VISUALIZATION_PROMPT}")
        print(f"--- Visualizer: Sending condensed task to the plotting agent ---")

        print(f"--- Sending this direct task to the agent ---\n{task_prompt}\n---------------------------------------------")

        system_prompt = ChatPromptTemplate.from_messages([
            ("system",
            "You are a Data Visualization Specialist. You have access to `eda_fact_sheet(df_path)` for initial recon and `python_repl_ast(query)` for generating and saving plots. "
            "The CSV is at this path: {df_path}. "
            "Your mission is to autonomously analyze the data and produce a series of 10 visualizations as instructed. "),
            ("human", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])

        visualizer_agent = create_tool_calling_agent(
            llm=self.llm_model,
            tools=[python_repl_ast],
            prompt=system_prompt
        )

        agent_executor = AgentExecutor(
            agent=visualizer_agent,
            tools=[python_repl_ast],
            verbose=True,
            # handle_parsing_errors=True,
            # return_intermediate_steps=True
        )

        max_attempts = 3
        attempt = 0
        while attempt < max_attempts:
            attempt += 1
            try:
                result = agent_executor.invoke({
                    "input": task_prompt,
                    "df_path": df_path,
                })

                final_output_string = result.get("output", "")

                parsed_output: VisualizationReport = fixing_parser.parse(final_output_string)

                summary_str = f"Successfully generated a report with {len(parsed_output.visualizations)} visualizations."

                return Command(
                    update={
                        "messages": [AIMessage(content=summary_str, name="Visualizer_node")],

                        "Visualizations": [{"final_answer": parsed_output.model_dump()}]
                    },
                    goto="supervisor",
                )

            except Exception as e:
                error_msg = (
                    f"Attempt {attempt} failed due to a parsing error: {str(e)}. "
                    f"You MUST provide a final answer that is a valid JSON object. Please strictly follow this schema: "
                    f"{VisualizationReport.model_json_schema()}"
                )
                print(f"--- Runtime/Parsing error encountered ---\n{error_msg}")
          
                task_prompt = f"here is the error from the last execution: {error_msg} so process the workflow accordingly" + "\n\n" + task_prompt

        return Command(
            update={
                "messages":  [
                    AIMessage(content="Error: The visualizer agent failed to produce a valid report after multiple attempts.",
                            name="Visualizer_node_Error")
                ],
                "Visualizations": [{"error": "Parsing failed after all retries"}]
            },
            goto="supervisor",
        )

"Data is about sales,provide the data overview along with the preprocessing steps needed to perform EDA" 

"""
Path = r"D:\Code Assistant\superstore sales.csv"
D:\Code Assistant\superstore sales.csv
"""

<>:563: SyntaxWarning: invalid escape sequence '\C'
<>:563: SyntaxWarning: invalid escape sequence '\C'
C:\Users\jayan\AppData\Local\Temp\ipykernel_496\3111309077.py:563: SyntaxWarning: invalid escape sequence '\C'
  """


'\nPath = r"D:\\Code Assistant\\superstore sales.csv"\nD:\\Code Assistant\\superstore sales.csv\n'

In [91]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import AgentExecutor, create_tool_calling_agent

In [92]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_groq import ChatGroq
# Load environment variables
load_dotenv()

# Get API key from .env
api_key = os.getenv("GOOGLE_API_KEY")

if not api_key:
    raise ValueError("GOOGLE_API_KEY not found. Make sure it's set in the .env file.")

# Initialize Gemini
llm_model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=api_key
)

# api_key = os.getenv("GROQ_API_KEY")

# if not api_key:
#     raise ValueError("GROQ_API_KEY not found. Make sure it's set in the .env file.")

# # Initialize the Groq Chat Model
# # The 120B model is identified as 'openai/gpt-oss-120b' on the Groq platform.
# llm_model = ChatGroq(
#     model_name="deepseek-r1-distill-llama-70b",
#     groq_api_key=api_key
# )

In [ ]:
import pandas as pd
import numpy as np
from langchain_core.tools import tool
from dotenv import load_dotenv
import sys,os
from io import StringIO
from pydantic import BaseModel, Field
import pandas as pd
from typing import Optional
import os

In [95]:
from langchain.tools import tool
from pydantic import BaseModel, Field
import pandas as pd, os, sys
from io import StringIO

class PandasExecutorInput(BaseModel):
    query: str = Field(..., description="A valid pandas code snippet that modifies 'df'.")
    path: str = Field(..., description="Full path to the dataset file (CSV, Excel, or Parquet).")

@tool(args_schema=PandasExecutorInput)
def python_repl_ast(query: str, path: str) -> str:
    """
    Executes a pandas command on the dataframe loaded from the given file path.
    Supports CSV, Excel, and Parquet. Saves the modified dataframe back.
    The dataframe is accessible as 'df' inside the code snippet.
    """
    # Normalize path and verify existence
    path = os.path.normpath(path)
    if not os.path.exists(path):
        return f"❌ File not found: {path}"

    # Load the dataframe
    ext = os.path.splitext(path)[-1].lower()
    try:
        if ext == ".csv":
            df = pd.read_csv(path, encoding='utf-8')
        elif ext in [".xls", ".xlsx"]:
            df = pd.read_excel(path)
        elif ext == ".parquet":
            df = pd.read_parquet(path)
        else:
            return f"❌ Unsupported file type: {ext}"
    except Exception as e:
        return f"❌ Error while reading file '{path}': {e}"

    # Prepare namespaces
    local_namespace = {"df": df, "pd": pd}
    global_namespace = {}

    old_stdout = sys.stdout
    sys.stdout = captured_output = StringIO()

    try:
        # Execute pandas code
        exec(query, global_namespace, local_namespace)
        output = captured_output.getvalue()

        # Save back modifications
        df = local_namespace.get("df", df)
        if ext == ".csv":
            df.to_csv(path, index=False, encoding='utf-8')
        elif ext in [".xls", ".xlsx"]:
            df.to_excel(path, index=False)
        elif ext == ".parquet":
            df.to_parquet(path, index=False)

        return f"✅ Executed successfully.\n{output or 'No print output.'}"
    except Exception as e:
        return f"❌ Execution failed: {e}"
    finally:
        sys.stdout = old_stdout


In [ ]:
from langchain_core.tools import tool

@tool
def eda_fact_sheet(path:str,sample_size: int = 3, freq_top: int = 3):
    """
    Generates a batchwise fact sheet for a dataset.

    Args:
        path (str): Path to CSV or Excel dataset.
        sample_size (int): Number of example values per column.
        freq_top (int): Number of top frequent values per column.

    Returns:
        dict: Fact sheet containing dataset-level info and column-level stats in batches.
    """
    path = path.strip().strip('"').strip("'")
    path = os.path.normpath(path)

    #--- Load dataset ---
    if path.endswith(".csv"):
        df = pd.read_csv(path)
    elif path.endswith(".xlsx") or path.endswith(".xls"):
        df = pd.read_excel(path)
    else:
        raise ValueError("Unsupported file format. Use CSV or Excel.")

    df = pd.read_csv(path)

    n_rows, n_columns = df.shape

    fact_sheet = {
        "n_rows": n_rows,
        "n_columns": n_columns,
        "batches": []
    }

    # --- Determine batch splits ---
    batch_size = 15
    batch_splits = [df.columns[i:i+batch_size].tolist() for i in range(0, n_columns, batch_size)]

    # --- Process each batch ---
    for batch_cols in batch_splits:
        batch_profile = {"columns": {}}
        for col in batch_cols:
            series = df[col]
            col_profile = {}

            total = len(series)

            # Core stats
            col_profile["dtype"] = str(series.dtype)
            col_profile["null_percent"] = round(float(series.isna().sum() / total * 100), 2)
            col_profile["unique_percent"] = round(float(series.nunique(dropna=True) / total * 100), 2)

            # Example values
            try:
                col_profile["examples"] = series.dropna().sample(
                    min(sample_size, series.dropna().shape[0]),
                    random_state=42
                ).tolist()
            except ValueError:
                col_profile["examples"] = []

            # Top frequent values
            if not series.isna().all():
                top_freq = series.value_counts(dropna=True).head(freq_top)
                col_profile["top_values"] = {str(k): int(v) for k, v in top_freq.to_dict().items()}

            # Numeric columns
            if pd.api.types.is_numeric_dtype(series):
                col_profile.update({
                    "min": float(series.min(skipna=True)),
                    "max": float(series.max(skipna=True)),
                    "mean": float(series.mean(skipna=True)),
                    "std": float(series.std(skipna=True))
                })
                if series.std(skipna=True) > 0:
                    z_scores = ((series - series.mean(skipna=True)) / series.std(skipna=True)).abs()
                    col_profile["has_outliers"] = bool((z_scores > 3).any())
                else:
                    col_profile["has_outliers"] = False

            # Datetime columns
            elif pd.api.types.is_datetime64_any_dtype(series):
                if not series.dropna().empty:
                    col_profile["min_date"] = str(series.min())
                    col_profile["max_date"] = str(series.max())

            # Text/categorical columns
            elif pd.api.types.is_object_dtype(series):
                lengths = series.dropna().astype(str).map(len)
                if not lengths.empty:
                    col_profile["avg_length"] = float(lengths.mean())
                    col_profile["max_length"] = int(lengths.max())
                unusual = series.dropna().astype(str).str.contains(r"[^a-zA-Z0-9\s]").sum()
                col_profile["unusual_char_percent"] = round(float(unusual / total * 100), 2)

            # Flags
            if series.nunique(dropna=True) == total:
                col_profile["is_identifier"] = True
            elif series.nunique(dropna=True) <= 1:
                col_profile["is_constant"] = True

            batch_profile["columns"][col] = col_profile

        fact_sheet["batches"].append(batch_profile)

    return fact_sheet


In [98]:
PreprocessingPlanner_prompt = """
You are Preprocessing Planner for EDA on sales datasets. 
Your sole goal is to output a direct, machine-readable cleaning plan so EDA runs smoothly and yields valid insights.
Call the eda_fact_sheet tool to understand the semantic meaning of each column present in the CSV.
Give necessary commands to ensure proper dtype, clean formats, EDA-ready columns with respect to the semantic meaning of the column. 
Make sure to not provide any destructive commands that will change the entire course of data.

---------------------------
🔑 EDA PREPROCESSING CHECKS (apply to each column logically):
1. Identifiers → convert numeric IDs to string; drop meaningless index cols (like 'Unnamed: 0'); drop duplicate rows.
2. Dates/Times → convert to datetime/time; drop invalid/future; extract year, month, day, weekday, hour.
3. Categoricals → strip whitespace/special chars; standardize case; merge rare categories to 'Other'.
4. Numeric Quantities → convert to int/float; fix negatives (set to abs unless return column); remove nonsensical zeros; handle outliers.
5. Monetary Values → remove currency symbols/commas; convert to float; fix negatives (refund ok); drop illogical zeros; derive Revenue/Profit/Margin if Quantity & Price exist.
6. Missing Data → fill categorical with 'Unknown', numeric with 0 or median; drop if >80% missing.
7. Duplicates → drop exact row duplicates; keep only necessary unique identifiers.
8. Consistency → verify Quantity×Price=Revenue; drop constant columns; ensure totals non-negative.
9. Sanity → ensure proper dtype, clean formats, EDA-ready columns.

OUTPUT_FORMAT (return only valid JSON):
{
  "plan":[
    {"column":"<column_name>","actions":"<chosen_actions>","Expected final form of the column":"<Final form of the column>"}
  ],
  "summary":"<One-sentence summary of preprocessing goal.>",
  "details":"<Step-by-step explanation of how the actions remove discrepancies for smooth EDA.>"
}
"""

In [99]:
Instructions = "Provide the plan for cleaning for performing EDA"

In [ ]:
Instructions = "Provide the plan for cleaning for performing EDA"

Path = r"D:\Code Assistant\avocado.csv"

In [ ]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import AgentExecutor, create_tool_calling_agent

task_prompt = (
    f"Find the instructions given by the user here : {Instructions} and follow this {PreprocessingPlanner_prompt} to the letter.modify in this path:{Path}"
)
# print(f"--- Sending this direct task to the agent ---\n{task_prompt}\n---------------------------------------------")

system_prompt = ChatPromptTemplate.from_messages([

("system",
    "You are a DataFrame cleaner consultant. Your primary tool is `eda_fact_sheet`. "
    "First, call the tool to get data insights. Then, based on the tool's output, "
    "provide a final answer formatted as a JSON object containing the preprocessing plan and summaries."),
("human", "{input}"),
MessagesPlaceholder(variable_name="agent_scratchpad")        
])

Analyzer_agent = create_tool_calling_agent(
    llm= llm_model,
    tools=[eda_fact_sheet],
    prompt=system_prompt
)

agent_executor = AgentExecutor(
    agent=Analyzer_agent,
    tools=[eda_fact_sheet],
    verbose=True,
    handle_parsing_errors=True,
    return_intermediate_steps=True
)

In [102]:
result = agent_executor.invoke({
                    "input": task_prompt,
                })



> Entering new AgentExecutor chain...

Invoking: `eda_fact_sheet` with `{'path': 'D:\\Code Assistant\\avocado.csv'}`


{'n_rows': 18249, 'n_columns': 14, 'batches': [{'columns': {'Unnamed: 0': {'dtype': 'int64', 'null_percent': 0.0, 'unique_percent': 0.29, 'examples': [6, 8, 48], 'top_values': {'0': 432, '1': 432, '2': 432}, 'min': 0.0, 'max': 52.0, 'mean': 24.232231903117977, 'std': 15.481044753757136, 'has_outliers': False}, 'Date': {'dtype': 'object', 'null_percent': 0.0, 'unique_percent': 0.93, 'examples': ['2018-02-11', '2015-11-01', '2016-01-24'], 'top_values': {'2015-12-27': 108, '2015-12-20': 108, '2015-12-13': 108}, 'avg_length': 10.0, 'max_length': 10, 'unusual_char_percent': 100.0}, 'AveragePrice': {'dtype': 'float64', 'null_percent': 0.0, 'unique_percent': 1.42, 'examples': [0.82, 0.97, 1.44], 'top_values': {'1.15': 202, '1.18': 199, '1.08': 194}, 'min': 0.44, 'max': 3.25, 'mean': 1.405978409775878, 'std': 0.40267655549555065, 'has_outliers': True}, 'Total Volume': {'dtyp

In [103]:
import json
output_string = result['output']
print("Type of output is:", type(output_string)) # This will print <class 'str'>

# 2. Parse the JSON string into a Python dictionary

# 3. NOW you can access the 'plan' key because output_data is a dictionary


Type of output is: <class 'str'>


In [104]:
print("Output string length:", len(output_string))
print("Output string preview:", output_string[:200])  # first 200 chars


Output string length: 3575
Output string preview: ```json
{
  "plan": [
    {
      "column": "Unnamed: 0",
      "actions": "Drop column",
      "Expected final form of the column": "Column removed"
    },
    {
      "column": "Date",
      "action


In [105]:
import json

# Remove leading/trailing backticks and whitespace
cleaned_output = output_string.strip("```json").strip("```").strip()

# Now parse it
output_data = json.loads(cleaned_output)

# Access the 'plan'
plan = output_data['plan']


In [106]:

# Example: print first item
print(plan)


[{'column': 'Unnamed: 0', 'actions': 'Drop column', 'Expected final form of the column': 'Column removed'}, {'column': 'Date', 'actions': 'Convert to datetime objects, extract year, month, day, and weekday into new columns.', 'Expected final form of the column': 'Datetime objects with additional columns for year, month, day, and weekday.'}, {'column': 'type', 'actions': 'Strip whitespace, convert to lowercase.', 'Expected final form of the column': "Cleaned categorical strings (e.g., 'conventional', 'organic')."}, {'column': 'region', 'actions': 'Strip whitespace, convert to title case.', 'Expected final form of the column': "Cleaned categorical strings (e.g., 'Albany', 'Atlanta')."}, {'column': 'AveragePrice', 'actions': 'No specific action beyond ensuring float dtype.', 'Expected final form of the column': 'Float values representing average price.'}, {'column': 'Total Volume', 'actions': 'No specific action beyond ensuring float dtype.', 'Expected final form of the column': 'Float va

In [107]:
import json

# --- New function using a standard for loop ---
def create_batches_with_loop(data_list, batch_size=4):
    """
    Groups items from a list into smaller sub-lists of a specified size using a for loop.
    """
    all_batches = []
    current_batch = []
    
    # Loop through each item in the original list
    for item in data_list:
        # Add the item to the batch we are currently building
        current_batch.append(item)
        
        # When the current batch reaches the desired size...
        if len(current_batch) == batch_size:
            # ...add it to our list of all batches...
            all_batches.append(current_batch)
            # ...and reset the current batch to start a new one.
            current_batch = []
            
    # After the loop, check if there are any leftover items in the last batch
    if current_batch:
        all_batches.append(current_batch)
        
    return all_batches

In [108]:
# Call the new function to create the list of batches.
# The result will be identical to the previous function.
batched_plan = create_batches_with_loop(plan, batch_size=4)

# --- Verification Step (this part remains the same) ---
# You can now directly use 'batched_plan'.
# For example, let's access and print the first batch to prove it's stored.

print("--- Verifying the final list of batches ---")
print(f"Total number of batches created: {len(batched_plan)}")
print("\n--- Content of the first batch (batched_plan[0]) ---")
# Using json.dumps for a clean print of the first batch
print(json.dumps(batched_plan[0], indent=4))

--- Verifying the final list of batches ---
Total number of batches created: 4

--- Content of the first batch (batched_plan[0]) ---
[
    {
        "column": "Unnamed: 0",
        "actions": "Drop column",
        "Expected final form of the column": "Column removed"
    },
    {
        "column": "Date",
        "actions": "Convert to datetime objects, extract year, month, day, and weekday into new columns.",
        "Expected final form of the column": "Datetime objects with additional columns for year, month, day, and weekday."
    },
    {
        "column": "type",
        "actions": "Strip whitespace, convert to lowercase.",
        "Expected final form of the column": "Cleaned categorical strings (e.g., 'conventional', 'organic')."
    },
    {
        "column": "region",
        "actions": "Strip whitespace, convert to title case.",
        "Expected final form of the column": "Cleaned categorical strings (e.g., 'Albany', 'Atlanta')."
    }
]


In [109]:
batched_plan

[[{'column': 'Unnamed: 0',
   'actions': 'Drop column',
   'Expected final form of the column': 'Column removed'},
  {'column': 'Date',
   'actions': 'Convert to datetime objects, extract year, month, day, and weekday into new columns.',
   'Expected final form of the column': 'Datetime objects with additional columns for year, month, day, and weekday.'},
  {'column': 'type',
   'actions': 'Strip whitespace, convert to lowercase.',
   'Expected final form of the column': "Cleaned categorical strings (e.g., 'conventional', 'organic')."},
  {'column': 'region',
   'actions': 'Strip whitespace, convert to title case.',
   'Expected final form of the column': "Cleaned categorical strings (e.g., 'Albany', 'Atlanta')."}],
 [{'column': 'AveragePrice',
   'actions': 'No specific action beyond ensuring float dtype.',
   'Expected final form of the column': 'Float values representing average price.'},
  {'column': 'Total Volume',
   'actions': 'No specific action beyond ensuring float dtype.',
 

In [110]:
# cleaner_prompt = """
# You are a methodical and precise data preprocessing execution agent. Your mission is to permanently modify a CSV file by executing a given JSON plan.

# ### TOOL BEHAVIOR (CRITICAL READ):
# Your only tool is `python_repl_ast(query, path)`. You must understand how it works:
# 1.  **Auto-Loading:** When you provide the `path`, the tool automatically loads the CSV into a pandas DataFrame named `df`.
# 2.  **Auto-Saving:** After your `query` successfully runs, the tool automatically saves the modified `df` back to the original `path`.
# 3.  **DO NOT LOAD OR SAVE:** Because of this, your `query` **MUST NOT** contain `pd.read_csv()` or `df.to_csv()`. The tool handles it. Your code should start directly with manipulating `df`.

# ### SYSTEMATIC EXECUTION PLAYBOOK:
# You will execute the plan iteratively in small, verifiable batches to ensure accuracy. Follow this playbook precisely:

# **1. Deconstruct the Plan:**
#    - Your mission is to execute the provided JSON plan. The plan is a list of actions.

# **2. Process in Batches:**
#    - Group the first 1 to 5 actions from the plan into a logical batch. Do not try to execute the entire plan at once.

# **3. Generate Batch Code & Self-Check:**
#    - For the current batch, write a single Python script (`query`) that performs all actions in the batch on the `df` DataFrame.
#    - For each action, you **MUST** refer to both the `"action"` and the `"Expected final form of the column"`. Your code's goal is to make the column match its expected final form.
#    - **CRITICAL SELF-CHECK:** After the transformation code for the batch, you MUST add a simple `print()` statement to verify the *last* or *most important* change in the batch. This is your self-check.
#      - For `convert_to_datetime`: `print(df['your_column'].dtype)`
#      - For `strip_whitespace`: `print(df['your_column'].iloc[0])`
#      - For `derive_revenue`: `print(df[['Revenue', 'Order Quantity']].head())`

# **4. Execute the Batch:**
#    - Call the `python_repl_ast` tool with the generated script for the current batch. Review the output of your `print()` statement to confirm success.

# **5. Repeat for Next Batch:**
#    - Repeat steps 2-4 for the next batch of 1-5 actions. Continue this process until all actions in the plan have been executed.

# ### CODING RULES (MANDATORY):
# - **ALWAYS** check for column existence before dropping or modifying: `if 'column_name' in df.columns:`.
# - **ALWAYS** use `errors='coerce'` when calling `pd.to_datetime` to prevent crashes from single invalid entries.

# ### FINAL OUTPUT FORMAT:
# After all batches have been successfully executed, your final answer MUST be a single, valid JSON object that strictly follows this schema. This should be your very last step.
# ```json
# {"summary": "A concise, one-sentence summary of the cleaning outcome.", "details": "A detailed summary of all actions taken and confirmation that all batches were successfully executed and saved."}"""


In [111]:
cleaner = """You are a data cleaner agent. Your primary tool is python_repl_ast.
Your task follows a strict two-phase process:
Phase 1: Execution
You will be given a plan to modify a dataset.
Execute all the necessary changes as described in the plan.
Phase 2: Verification and Completion
After applying all changes, you must perform a self-check to verify that the modifications were successful. This involves running verification code (e.g., using df.info(), df.describe(), or checking for nulls/unique values) to confirm the data matches its expected final form.
Once you have confirmed through the self-check that all changes were applied correctly, the project is considered complete.
Your final action is to provide the output in the specified JSON format. Do not generate this final output until after the self-check is successfully completed.Give the final output as the below form
{"summary": "A concise, one-sentence summary of the cleaning outcome.", 
"details": "A detailed summary of all actions taken and confirmation that all batches were successfully executed and saved."}"""


In [112]:

system_prompt = ChatPromptTemplate.from_messages(
[
(
    "system",
    "You are a DataFrame cleaner agent. You have access to a tool `python_repl_ast(query)` to modify the dataframe CSV/DataFrame."
    "Follow the instructions here : {cleaner} and  in the input to the letter and make the necessary changes to the dataframe."
),
("human", "{input}"),
MessagesPlaceholder(variable_name="agent_scratchpad")        
]
)
Cleaner_agent = create_tool_calling_agent(
    llm=llm_model,
    tools=[python_repl_ast],
    prompt=system_prompt
)

agent_executor = AgentExecutor(
    agent=Cleaner_agent,
    tools=[python_repl_ast],
    verbose=True,
    handle_parsing_errors=True,
    return_intermediate_steps=True
)

In [113]:
results = []

for i, batch in enumerate(batched_plan, start=1):
    # Convert batch to a string to pass into the prompt
    clean_plan = str(batch)
    task_prompt = f"Apply the following cleaning plan (batch {i} of {len(batched_plan)}) to the dataset at path: {Path}\nPlan details:\n{clean_plan}"
    
    # Invoke the agent
    result = agent_executor.invoke({"input": task_prompt,"cleaner":cleaner})
    
    # Store result for review
    results.append(result)

# Optional: review all summaries
for idx, r in enumerate(results, start=1):
    print(f"Batch {idx} summary:", r.get("summary"))
    print(f"Details:", r.get("details"))



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df = df.drop(columns=['Unnamed: 0'])\ndf['Date'] = pd.to_datetime(df['Date'])\ndf['Year'] = df['Date'].dt.year\ndf['Month'] = df['Date'].dt.month\ndf['Day'] = df['Date'].dt.day\ndf['Weekday'] = df['Date'].dt.weekday\ndf['type'] = df['type'].str.strip().str.lower()\ndf['region'] = df['region'].str.strip().str.title()", 'path': 'D:\\Code Assistant\\avocado.csv'}`


✅ Executed successfully.
No print output.
Invoking: `python_repl_ast` with `{'query': "print(df.info())\nprint(df[['Date', 'Year', 'Month', 'Day', 'Weekday', 'type', 'region']].head())", 'path': 'D:\\Code Assistant\\avocado.csv'}`


✅ Executed successfully.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18249 entries, 0 to 18248
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          18249 non-null  object 
 1   AveragePrice  18249 non-null  float64
 2   Tota

In [69]:
result

{'input': 'Apply the following cleaning plan (batch 2 of 2) to the dataset at path: D:\\Code Assistant\\Bakery sale.csv\nPlan details:\n[{\'column\': \'article\', \'actions\': "Strip whitespace and special characters. Standardize case to title case. Merge rare categories (less than 1% frequency) into \'Other\'.", \'Expected final form of the column\': "Cleaned strings, title-cased, with rare categories consolidated to \'Other\'."}, {\'column\': \'Quantity\', \'actions\': \'Convert to integer. Convert negative values to their absolute value. Set nonsensical zeros to NaN. Handle outliers (e.g., cap at a reasonable upper bound or replace with median/mean if appropriate for sales data).\', \'Expected final form of the column\': \'Integer (e.g., 1, 2, 3), with positive values, and outliers handled.\'}, {\'column\': \'unit_price\', \'actions\': "Remove currency symbols (\'€\') and commas (\',\'). Convert to float. Fix negative values by taking the absolute value (assuming refunds are accepta